<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Trábajo Práctico Nro. 2 - Custom embedddings con Gensim
## Alumno: Ezequiel Alejandro Caamaño
## a1802



## Consignas
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [3]:
!pip install gensim
!pip install tensorflow
!pip install keras

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset textos del projecto Gutember.

In [5]:
# Descargar la carpeta de dataset
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/ezequiel/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ezequiel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ezequiel/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
from nltk.corpus import gutenberg

# Ver qué archivos de texto están disponibles
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [7]:
# Elegir una obra específica de Shakespeare
file_id = 'shakespeare-hamlet.txt'
author = 'William Shakespeare'

# Obtener texto y dividir en oraciones
raw_text = gutenberg.raw(file_id)
# Separar por saltos de línea
lines = raw_text.split('\n')

# Crear DataFrame
df = pd.DataFrame(lines, columns=['line'])

# Mostrar las primeras filas
df.head()


,line
0,[The Tragedie of Hamlet by William Shakespeare...
1,
2,
3,Actus Primus. Scoena Prima.
4,


In [8]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 4923


### 1 - Preprocesamiento

In [9]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# Lista donde guardamos las secuencias de palabras
line_tokens = []

# Recorrer todas las líneas del DataFrame
for _, row in df.iterrows():
    line_tokens.append(text_to_word_sequence(row['line']))

In [10]:
# Demos un vistazo
line_tokens[:2]

[['the', 'tragedie', 'of', 'hamlet', 'by', 'william', 'shakespeare', '1599'],
 []]

### 2 - Crear los vectores (word2vec)

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [13]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(line_tokens)

In [14]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 4923


In [15]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 686


### 3 - Entrenar embeddings

In [16]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(line_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 214488.5625
Loss after epoch 1: 125322.65625
Loss after epoch 2: 124484.25
Loss after epoch 3: 125573.21875
Loss after epoch 4: 126025.875
Loss after epoch 5: 126054.25
Loss after epoch 6: 125988.4375
Loss after epoch 7: 119494.375
Loss after epoch 8: 108546.25
Loss after epoch 9: 109482.75
Loss after epoch 10: 109111.125
Loss after epoch 11: 107289.0
Loss after epoch 12: 107214.75
Loss after epoch 13: 106225.375
Loss after epoch 14: 104971.25
Loss after epoch 15: 105226.875
Loss after epoch 16: 104648.25
Loss after epoch 17: 95934.75
Loss after epoch 18: 89775.0
Loss after epoch 19: 88752.25
Loss after epoch 20: 89097.0
Loss after epoch 21: 89044.0
Loss after epoch 22: 87866.75
Loss after epoch 23: 87344.5
Loss after epoch 24: 87375.75
Loss after epoch 25: 87695.25
Loss after epoch 26: 87623.0
Loss after epoch 27: 86140.5
Loss after epoch 28: 85188.0
Loss after epoch 29: 86250.0
Loss after epoch 30: 86138.75
Loss after epoch 31: 85521.0
Loss after epoch 32: 84959.0

(763228, 1484900)

### 4 - Ensayar

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["king"], topn=10)

[('gertrude', 0.6269384026527405),
 ("do's", 0.6220626831054688),
 ('polonius', 0.6008369326591492),
 ('sister', 0.5973669290542603),
 ('guildensterne', 0.5962295532226562),
 ('rosincrance', 0.5945658683776855),
 ('osricke', 0.5821160078048706),
 ('lords', 0.5769570469856262),
 ('vnto', 0.5703410506248474),
 ('sometimes', 0.5670945644378662)]

In [18]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["king"], topn=10)

[('had', -0.0628107339143753),
 ('whose', -0.06991232931613922),
 ('armes', -0.07748594880104065),
 ('indeed', -0.08991096913814545),
 ('was', -0.09926942735910416),
 ('could', -0.10122586041688919),
 ('beene', -0.11083291471004486),
 ('sir', -0.11967476457357407),
 ('take', -0.12293863296508789),
 ('old', -0.12380380928516388)]

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["hamlet"], topn=10)

[('barnardo', 0.655182957649231),
 ('fortinbras', 0.6541325449943542),
 ('adue', 0.6508650183677673),
 ('polonius', 0.6418254375457764),
 ('osricke', 0.6356576681137085),
 ('marcellus', 0.6117737293243408),
 ('exit', 0.6076095700263977),
 ('exeunt', 0.595268189907074),
 ('slaine', 0.5761756300926208),
 ('young', 0.5588536858558655)]

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sword"], topn=5)

[('braines', 0.7698466777801514),
 ('face', 0.6928442716598511),
 ('lookes', 0.686236560344696),
 ('wholsome', 0.6793888807296753),
 ('mouth', 0.6696476936340332)]

In [21]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [22]:
# el método `get_vector` permite obtener los vectores:
vector_king = w2v_model.wv.get_vector("king")
print(vector_king)

[-1.17729649e-01  8.73767734e-02  2.40540996e-01 -2.96575308e-01
 -6.30337074e-02 -2.20735639e-01  1.63187101e-01  2.94804901e-01
 -4.23564196e-01  7.70663284e-03  3.12163144e-01 -8.07771310e-02
  3.02716382e-02  6.07665442e-02  8.92783031e-02  4.10832502e-02
  2.83607692e-01 -3.70007336e-01 -1.73829451e-01 -1.09379672e-01
 -2.27814689e-01 -1.07470732e-02 -3.77905294e-02 -8.59440677e-03
  2.87100747e-02  4.41186607e-01 -1.65451810e-01 -1.79138258e-01
 -3.95663008e-02  1.81369975e-01 -3.07795014e-02 -1.23164177e-01
  2.17003655e-02  8.92642438e-02 -2.04962030e-01  2.25201204e-01
 -3.62459645e-02 -1.11998692e-01  2.47934997e-01 -2.78438419e-01
  1.21790245e-01 -2.45886624e-01  2.13732332e-01  2.19021603e-01
  4.24427390e-01  3.00516278e-01 -8.98588300e-02  3.99233252e-02
  2.07050547e-01 -6.92798570e-02 -9.17193815e-02  9.35325548e-02
 -4.23813574e-02  1.51469618e-01 -3.03153098e-01  7.90440515e-02
 -1.67809844e-01  1.04287736e-01  9.14071277e-02 -2.98460513e-01
  3.61997895e-02  1.81821

In [23]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_king)

[('king', 1.0),
 ('gertrude', 0.6269382238388062),
 ("do's", 0.6220626831054688),
 ('polonius', 0.6008369326591492),
 ('sister', 0.5973668694496155),
 ('guildensterne', 0.5962294936180115),
 ('rosincrance', 0.5945658087730408),
 ('osricke', 0.5821160078048706),
 ('lords', 0.5769570469856262),
 ('vnto', 0.5703409910202026)]

In [24]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["king"], topn=10)

[('gertrude', 0.6269384026527405),
 ("do's", 0.6220626831054688),
 ('polonius', 0.6008369326591492),
 ('sister', 0.5973669290542603),
 ('guildensterne', 0.5962295532226562),
 ('rosincrance', 0.5945658683776855),
 ('osricke', 0.5821160078048706),
 ('lords', 0.5769570469856262),
 ('vnto', 0.5703410506248474),
 ('sometimes', 0.5670945644378662)]

### 5 - Visualizar agrupación de vectores

In [25]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [26]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [27]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

### Conclusiones

En este trabajo en particular se eligio el corpus de la obra "Hamlet" de William Shakespeare. Resultó interesante el analisis pues al observar los resultados de la busqueda de similitudes con vectores y embeddings obtenidos se puede apreciar que en muchos casos se asocian con nombres de personajes de la obra. Al tratar de entender más allá de lo computacional se entiende que al tratarse de una obra de teatro, gran parte, si no todo el corpus está compusto por diálogos y las referencias permanente a los personajes. Por eso se pueden encontrar vectores como los de "Laertes", "Horatio", a "Hamlet", o el mismo vector "ghost"y "father", que en la obra acompañan en gran parte de los actos al personaje principal que la titula. Se aprecia como rompe el flujo la busqueda de una palabra que no forma parte del conjunto analizado, lo cual era de esperar. 
